In [1]:
import cv2
import numpy as np
from PIL import Image, ImageFilter

In [2]:
# Read the video from specified path
cam = cv2.VideoCapture("./new/encryptedvideo.avi")
# frames = cam.get(cv2.CAP_PROP_FRAME_COUNT)
fps = cam.get(cv2.CAP_PROP_FPS)
# seconds = round(frames / fps)
print(fps)

30.0


In [3]:
image='./SquareDetectionResults/'
ext='.bmp'

In [4]:
from PIL import Image
import os
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import random
from math import log
from tqdm import tqdm

In [5]:
def getImageMatrix(imageName):
    im = Image.open(imageName, mode="r", formats=None) 
    pix = im.load()
    color = 1
    if type(pix[0,0]) == int:
        color=0
    image_size = im.size 
    image_matrix = []
    for width in range(int(image_size[0])):
        row = []
        for height in range(int(image_size[1])):
                row.append((pix[width,height]))
        image_matrix.append(row)
    return image_matrix,image_size[0],color

In [6]:
def LogisticDecryption(imageName, key):
    N = 256
    key_list = [ord(x) for x in key]
    G = [key_list[0:4] ,key_list[4:8], key_list[8:12]]
    g = []
    R = 1
    for i in range(1,4):
        s = 0
        for j in range(1,5):
            s += G[i-1][j-1] * (10**(-j))
        g.append(s)
        R = (R*s) % 1
    
    L_x = (R + key_list[12]/256) % 1
    S_x = round(((g[0]+g[1]+g[2])*(10**4) + L_x *(10**4)) % 256)
    V1 = sum(key_list)
    V2 = key_list[0]
    for i in range(1,13):
        V2 = V2 ^ key_list[i]
    V = V2/V1

    L_y = (V+key_list[12]/256) % 1
    S_y = round((V+V2+L_y*10**4) % 256)
    C1_0 = S_x
    C2_0 = S_y
    
    C = round((L_x*L_y*10**4) % 256)
    I_prev = C
    I_prev_r = C
    I_prev_g = C
    I_prev_b = C
    I = C
    I_r = C
    I_g = C
    I_b = C
    x_prev = 4*(S_x)*(1-S_x)
    y_prev = 4*(L_x)*(1-S_y)
    x = x_prev
    y = y_prev
    imageMatrix, dimension, color = getImageMatrix(imageName)
    henonDecryptedImage = []
    for i in range(dimension):
        row = []
        for j in range(dimension):
            while x <0.8 and x > 0.2 :
                x = 4*x*(1-x)
            while y <0.8 and y > 0.2 :
                y = 4*y*(1-y)
            x_round = round((x*(10**4))%256)
            y_round = round((y*(10**4))%256)
            C1 = x_round ^ ((key_list[0]+x_round) % N) ^ ((C1_0 + key_list[1])%N)
            C2 = x_round ^ ((key_list[2]+y_round) % N) ^ ((C2_0 + key_list[3])%N) 
            if color:
                I_r = ((((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((I_prev_r + key_list[7]) % N) ^ imageMatrix[i][j][0]) + N-key_list[6])%N
                I_g = ((((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((I_prev_g + key_list[7]) % N) ^ imageMatrix[i][j][1]) + N-key_list[6])%N
                I_b = ((((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((I_prev_b + key_list[7]) % N) ^ imageMatrix[i][j][2]) + N-key_list[6])%N
                I_prev_r = imageMatrix[i][j][0]
                I_prev_g = imageMatrix[i][j][1]
                I_prev_b = imageMatrix[i][j][2]
                row.append((I_r,I_g,I_b))
                x = (x +  imageMatrix[i][j][0]/256 + key_list[8]/256 + key_list[9]/256) % 1
                y = (x +  imageMatrix[i][j][0]/256 + key_list[8]/256 + key_list[9]/256) % 1  
            else:
                I = ((((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((I_prev+key_list[7]) % N) ^ imageMatrix[i][j]) + N-key_list[6])%N
                I_prev = imageMatrix[i][j]
                row.append(I)
                x = (x +  imageMatrix[i][j]/256 + key_list[8]/256 + key_list[9]/256) % 1
                y = (x +  imageMatrix[i][j]/256 + key_list[8]/256 + key_list[9]/256) % 1
            for ki in range(12):
                key_list[ki] = (key_list[ki] + key_list[12]) % 256
                key_list[12] = key_list[12] ^ key_list[ki]
        henonDecryptedImage.append(row)
    if color:
        im = Image.new("RGB", (dimension, dimension))
    else: 
        im = Image.new("L", (dimension, dimension)) # L is for Black and white pixels
    pix = im.load()
    for x in range(dimension-2):
        for y in range(dimension-2):
            pix[x, y] = henonDecryptedImage[x][y]
    im.save(image+"decryp"+imageName[28:-4]+ext, "BMP")

In [7]:
key = "abcdefghijklm"

In [8]:
folder_dir = "C:/Users/USER/Downloads/Secure-Sharing-of-CCTV-footage-master (2)/Secure-Sharing-of-CCTV-footage-master/SquareDetectionResults/encrypted_frames/"
cropped_images = []
images = os.listdir(folder_dir)
images.sort()
print(images)
cnt=0
for image in images:
    # Open image and make into Numpy array
    print(folder_dir+'/'+image)
    im_folder=folder_dir+'/'+image
    
    imidk = Image.open(im_folder)#cv2.imread(im_folder)
    
    
    na = np.array( Image.open(im_folder))
    orig = na.copy()    # Save original
    im = cv2.cvtColor(np.array( Image.open(im_folder)), cv2.COLOR_RGB2BGR)
    #cv2.imshow("idk what", orig)
    #cv2.waitKey(5000)
    # cv2.imwrite("./im.png", im)
    #cv2.imwrite("./idk what.png", orig)
    # Median filter to remove outliers
    # im = im.filter(ImageFilter.MedianFilter(3))
    # Find X,Y coordinates of all yellow pixels
    na = np.array(im)
    orig = na.copy()
    yellowY, yellowX = np.where(np.all(na==[255, 0, 0],axis=2))
    #im = im.filter(ImageFilter.MedianFilter(3))
    # Find X,Y coordinates of all yellow pixels
    #yellowY2, yellowX2= np.where(np.all(na==[0, 0, 255],axis=-2))
    zipped = np.column_stack((yellowX,yellowY))
    uniqueY = list(yellowY)
    uniqueX= list(set(yellowX))

    uniquesetX={uniqueX[0]}
    uniquesetY={uniqueY[0]}
    for x in range(0,len(uniqueX)-1):
        if(uniqueX[x+1]-uniqueX[x]>=2):
            uniquesetX.add(uniqueX[x])
            uniquesetX.add(uniqueX[x+1])
    uniquesetX.add(uniqueX[len(uniqueX)-1])
    uniquesetX=sorted(uniquesetX)
    uniquesetY=list()
    for a in uniquesetX:
        for x,y in zipped:
            if(a==x):
                uniquesetY.append(y)
    uniqueset2Y=list()
    uniqueset2Y.append(uniquesetY[0])
    print(uniqueset2Y)
    for x in range(0,len(uniquesetY)-1):
        if(uniquesetY[x+1]-uniquesetY[x]!=1):
            uniqueset2Y.append(uniquesetY[x])
            uniqueset2Y.append(uniquesetY[x+1])
    uniqueset2Y.append(uniquesetY[len(uniquesetY)-1])
    print("uniqesetY",uniqueset2Y)
    uniqueset3Y=list()
    for x in range(0,len(uniqueset2Y),4):
        uniqueset3Y.append(uniqueset2Y[x])
        uniqueset3Y.append(uniqueset2Y[x+1])
    print(uniqueset3Y)
    img = cv2.imread('./encrypted_frames/'+image)
    faces=[]
    
    for i in range(0,len(uniquesetX)-1,2):
        a=uniqueset3Y[i]
        b=uniqueset3Y[i+1]
        c=uniquesetX[i]
        d=uniquesetX[i+1]
        #print(a,b,c,d)
        face=img[a+1:b,c+1:d]
        filename=image+ext
        cv2.imwrite(filename, face)
        LogisticDecryption(filename, key)
        filename=image+"decryp"+ext
        DecryptResult=cv2.imread(filename)
        img[a+1:b,c+1:d]=DecryptResult
    if cnt<10:
        fname='./SquareDetectionResults/decrypted_frames/decrypt0'+str(cnt)
    else:
        fname='./SquareDetectionResults/decrypted_frames/decrypt'+str(cnt)
    cv2.imwrite(fname+ext, img)
    cnt+=1


['EncryptionResult00.png', 'EncryptionResult01.png', 'EncryptionResult02.png', 'EncryptionResult03.png', 'EncryptionResult04.png', 'EncryptionResult05.png', 'EncryptionResult06.png', 'EncryptionResult07.png', 'EncryptionResult08.png', 'EncryptionResult09.png', 'EncryptionResult10.png', 'EncryptionResult11.png', 'EncryptionResult12.png', 'EncryptionResult13.png', 'EncryptionResult14.png', 'EncryptionResult15.png', 'EncryptionResult16.png', 'EncryptionResult17.png', 'EncryptionResult18.png', 'EncryptionResult19.png', 'EncryptionResult20.png', 'EncryptionResult21.png', 'EncryptionResult22.png', 'EncryptionResult23.png', 'EncryptionResult24.png', 'EncryptionResult25.png', 'EncryptionResult26.png', 'EncryptionResult27.png', 'EncryptionResult28.png', 'EncryptionResult29.png', 'EncryptionResult30.png', 'EncryptionResult31.png', 'EncryptionResult32.png', 'EncryptionResult33.png', 'EncryptionResult34.png', 'EncryptionResult35.png', 'EncryptionResult36.png', 'EncryptionResult37.png', 'Encryption

IndexError: list index out of range

In [ ]:
# importing libraries
import os
import cv2
from PIL import Image
import os.path
import shutil
# Checking the current directory path
print(os.getcwd())

# Folder which contains all the images
# from which video is to be generated

os.chdir("C:/Users/USER/Downloads/Secure-Sharing-of-CCTV-footage-master (2)/Secure-Sharing-of-CCTV-footage-master/SquareDetectionResults/decrypted_frames")
path = "C:/Users/USER/Downloads/Secure-Sharing-of-CCTV-footage-master (2)/Secure-Sharing-of-CCTV-footage-master/SquareDetectionResults/decrypted_frames"

mean_height = 0
mean_width = 0
num_of_images = len(os.listdir('.'))
print(num_of_images)
substr="decr"
for file in os.listdir('.'):
    if(substr not in file):
        print("f")
        continue
   #print("y")
    im = Image.open(os.path.join(path, file))
    width, height = im.size
    mean_width += width
    mean_height += height
	# im.show() # uncomment this for displaying the image

# Finding the mean height and width of all images.
# This is required because the video frame needs
# to be set with same width and height. Otherwise
# images not equal to that width height will not get
# embedded into the video
mean_width = int(mean_width / num_of_images)
mean_height = int(mean_height / num_of_images)

# print(mean_height)
# print(mean_width)

# Resizing of the images to give
# them same width and height
for file in os.listdir('.'):
    if(substr not in file):
        print("f")
        continue
    if file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith("png"):
        im = Image.open(os.path.join(path, file))

		# im.size includes the height and width of image
        width, height = im.size
        print(width, height)

		# resizing
        imResize = im.resize((mean_width, mean_height), Image.ANTIALIAS)
        imResize.save( file, 'JPEG', quality = 95) # setting quality
		# printing each resized image name
        print(im.filename.split('/')[-1], " is resized")


# Video Generating function
def generate_video():
    image_folder = "C:/Users/USER/Downloads/Secure-Sharing-of-CCTV-footage-master (2)/Secure-Sharing-of-CCTV-footage-master/SquareDetectionResults/decrypted_frames" # make sure to use your folder
    video_name = 'decryptedvideo.avi'
    os.chdir("C:/Users/USER/Downloads/Secure-Sharing-of-CCTV-footage-master (2)/Secure-Sharing-of-CCTV-footage-master/new")

    images = [img for img in os.listdir(image_folder) if img.endswith(".bmp") or img.endswith(".jpg") or img.endswith("png")]
    images.sort()
    print(images)

    frame = cv2.imread(os.path.join(image_folder, images[0]))

	# setting the frame width, height width
	# the width, height of first image
    height, width, layers = frame.shape

    video = cv2.VideoWriter(video_name, 0, fps, (width, height))

	# Appending the images to the video one by one
    for image in images:
        video.write(cv2.imread(os.path.join(image_folder, image)))
	# Deallocating memories taken for window creation
    cv2.destroyAllWindows()
    video.release() # releasing the video generated


# Calling the generate_video function
generate_video()